In [1]:
from bs4 import BeautifulSoup

html = r'X:\Misinfo\HTML\real\3.html'

with open(html, encoding='utf8') as f:
    soup = BeautifulSoup(f, 'html.parser')
f.close()

## Corresponding to all_meta_combined.ipynb

Create dictionary of site, label, and all meta tags combined

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from pathlib import Path
import numpy as np
import pickle
from tqdm import tqdm

# Get url keys
with open('real_url_key.pkl', 'rb') as f:
    real_url_key = pickle.load(f)
f.close()
with open('fake_url_key.pkl', 'rb') as f:
    fake_url_key = pickle.load(f)
f.close()

directory1 = Path.cwd() / 'HTML' / 'real'
directory2 = Path.cwd() / 'HTML' / 'fake'
directory = [directory1, directory2]

dict_ = {'index' : [],
        'label' : [],
        'site' : [],
        'meta_content_combined' : []}

# The first loop creates the name list, the second loop creates vectors for each site corresponding to the number of each
num_without_content = 0
for dir_ in tqdm(directory):
    for html in os.listdir(dir_):
        # Open html file as soup object
        with open(str(dir_) + '/' + html, encoding='utf8') as f:
            soup = BeautifulSoup(f, 'html.parser')
        f.close()

        # Populate dict
        index = int(str(html).split('.')[0])
        label = str(dir_)[-4:]
        site = real_url_key[index] if str(dir_)[-4:] == 'real' else fake_url_key[index]

        metatags = soup.find_all('meta')
        content = ''
        for tag in metatags:
            try:
                content += ' ' + tag.attrs['content']
            except:
                try:
                    content += ' ' + tag.attrs['value']
                except:
                    continue
            
            
        
        dict_['index'].append(index)
        dict_['label'].append(label)
        dict_['site'].append(site)
        dict_['meta_content_combined'].append(content)

print('\nThere are {} sites without meta content'.format(num_without_content))

save to csv

In [12]:
df = pd.DataFrame(dict_)
df_csv = pd.read_csv('html_data_expanded.csv')
joined = df_csv.merge(df, on=['index', 'label'])
joined.rename(columns={'Column1' : 'global_index', 'site_x' : 'site'}, inplace=True)
joined.drop(columns=['site_y'], inplace=True)
joined.to_csv('all_meta_combined.csv')

' text/html; charset=UTF-8 width=device-width,initial-scale=1 A+b/H0b8RPXNaJgaNFpO0YOFuGK6myDQXlwnJB3SwzvNMfcndat4DZYMrP4ClJIzYWo3/yP2S+8FTZ/lpqbPAAEAAABueyJvcmlnaW4iOiJodHRwczovL2ltYXNkay5nb29nbGVhcGlzLmNvbTo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzVGhpcmRQYXJ0eSI6dHJ1ZX0= A9ZgbRtm4pU3oZiuNzOsKcC8ppFSZdcjP2qYcdQrFKVzkmiWH1kdYY1Mi9x7G8+PS8HV9Ha9Cz0gaMdKsiVZIgMAAAB7eyJvcmlnaW4iOiJodHRwczovL2RvdWJsZWNsaWNrLm5ldDo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9 AxL6oBxcpn5rQDPKSAs+d0oxNyJYq2/4esBUh3Yx5z8QfcLu+AU8iFCXYRcr/CEEfDnkxxLTsvXPJFQBxHfvkgMAAACBeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZXRhZ3NlcnZpY2VzLmNvbTo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9 A9KPtG5kl3oLTk21xqynDPGQ5t18bSOpwt0w6kGa6dEWbuwjpffmdUpR3W+faZDubGT+KIk2do0BX2ca16x8qAcAAACBeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZXN5bmRpY2F0aW9uLmNvbTo0NDMiLCJmZWF0dXJlIjoiVHJ1c3R

## To get list of different meta tags

In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from pathlib import Path
import numpy as np
import pickle
from tqdm import tqdm

# Get url keys
with open('real_url_key.pkl', 'rb') as f:
    real_url_key = pickle.load(f)
f.close()
with open('fake_url_key.pkl', 'rb') as f:
    fake_url_key = pickle.load(f)
f.close()

directory1 = Path.cwd() / 'HTML' / 'real'
directory2 = Path.cwd() / 'HTML' / 'fake'
directory = [directory1, directory2]
# Will hold all the potential meta "name" categories
# non case sensitive
name_list = []

dict_ = {'index' : [],
        'label' : [],
        'site' : [],
        'meta_names' : []}

# The first loop creates the name list, the second loop creates vectors for each site corresponding to the number of each
for dir_ in tqdm(directory):
    for html in os.listdir(dir_):
        # Open html file as soup object
        with open(str(dir_) + '/' + html, encoding='utf8') as f:
            soup = BeautifulSoup(f, 'html.parser')
        f.close()

        metatags = soup.find_all('meta')
        for tag in metatags:
            try:
                name_list.append(tag.attrs['name'])
            except:
                continue

name_list = list(set(name_list))

meta_name_vocab = {}
for i in range(len(name_list)):
    meta_name_vocab[i] = name_list[i]

with open('meta_name_vocab.pkl', 'wb') as f:
    pickle.dump(meta_name_vocab, f)
f.close()


# df = pd.DataFrame(dict_)


100%|██████████| 2/2 [04:18<00:00, 129.40s/it]


In [14]:
dict_ = {'index' : [],
        'label' : [],
        'site' : [],
        'name_vector' : []}

meta_name_dict_default = {}
for name in name_list:
    meta_name_dict_default[name] = 0

for dir_ in tqdm(directory):
    for html in os.listdir(dir_):
        # Construct Dictionary
        meta_name_dict = meta_name_dict_default.copy()

        # Open html file as soup object
        with open(str(dir_) + '/' + html, encoding='utf8') as f:
            soup = BeautifulSoup(f, 'html.parser')
        f.close()

        index = int(str(html).split('.')[0])
        label = str(dir_)[-4:]
        site = real_url_key[index] if str(dir_)[-4:] == 'real' else fake_url_key[index]

        # Get metatags and add them to meta name dict
        metatags = soup.find_all('meta')
        for tag in metatags:
            try:
                meta_name_dict[tag.attrs['name']] += 1
            except:
                continue
        
        # Create vector from meta name dict
        name_vector = list(meta_name_dict.values())

        dict_['index'].append(index)
        dict_['label'].append(label)
        dict_['site'].append(site)
        dict_['name_vector'].append(name_vector)

df = pd.DataFrame(dict_)


100%|██████████| 2/2 [01:46<00:00, 53.42s/it]


In [27]:
df_csv = pd.read_csv('html_data_expanded.csv')
joined = df_csv.merge(df, on=['index', 'label'])
joined.rename(columns={'Column1' : 'global_index', 'site_x' : 'site'}, inplace=True)
joined.drop(columns=['site_y'], inplace=True)
joined.to_csv('html_data.csv')

## To get original meta tags


In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from pathlib import Path
import numpy as np
import pickle
from tqdm import tqdm

# Get url keys
with open('real_url_key.pkl', 'rb') as f:
    real_url_key = pickle.load(f)
f.close()
with open('fake_url_key.pkl', 'rb') as f:
    fake_url_key = pickle.load(f)
f.close()

directory1 = Path.cwd() / 'HTML' / 'real'
directory2 = Path.cwd() / 'HTML' / 'fake'

directory = [directory1, directory2]
dict_ = {'index' : [],
        'label' : [],
        'site' : [],
        'og:title' : [],
        'og:keywords' : [],
        'og:description' : [],
        'twitter:title' : [],
        'twitter:keywords' : [],
        'twitter:description' : [],
        'keywords': [],
        'description' : []}

for dir_ in tqdm(directory):
    for html in os.listdir(dir_):
        # Open html file as soup object
        with open(str(dir_) + '/' + html, encoding='utf8') as f:
            soup = BeautifulSoup(f, 'html.parser')
        f.close()

        # metadata
        index = int(str(html).split('.')[0])
        label = str(dir_)[-4:]
        site = real_url_key[index] if str(dir_)[-4:] == 'real' else fake_url_key[index]
        
        # ---------------------------------------------------------------------------- #
        #                             open graph meta tags                             #
        # ---------------------------------------------------------------------------- #
        
        # og:title
        title = soup.find("meta", property="og:title")
        try:
            og_title = title['content'] if title else np.nan
        except KeyError as err:
            og_title = np.nan
            print('KeyError at index {} title'.format(index))
            print('continuing...')
        
        # og:keywords
        og_keywords = soup.find('meta', property='og:keywords')
        try:
            og_keywords = og_keywords['content'] if og_keywords else np.nan
        except KeyError as err:
            og_keywords = np.nan
            print('KeyError at index {} og:keywords'.format(index))
            print('continuing...')
        
        # og:description
        og_description = soup.find('meta', property='og:description')
        try:
            og_description = og_description['content'] if og_description else np.nan
        except KeyError as err:
            og_description = np.nan
            print('KeyError at index {} og:description'.format(index))
            print('continuing...')


        # ---------------------------------------------------------------------------- #
        #                               twitter meta tags                              #
        # ---------------------------------------------------------------------------- #
        
        # twitter:title
        tw_title = soup.find("meta", attrs={'name':"twitter:title"})
        try:
            tw_title = tw_title['content'] if tw_title else np.nan
        except KeyError as err:
            tw_title = np.nan
            print('KeyError at index {} twitter:title'.format(index))
            print('continuing...')
        
        # twitter:keywords
        tw_keywords = soup.find('meta', attrs={'name':"twitter:keywords"})
        try:
            tw_keywords = tw_keywords['content'] if tw_keywords else np.nan
        except KeyError as err:
            tw_keywords = np.nan
            print('KeyError at index {} twitter:keywords'.format(index))
            print('continuing...')
        
        # twitter:description
        tw_description = soup.find('meta', attrs={'name':"twitter:description"})
        try:
            tw_description = tw_description['content'] if tw_description else np.nan
        except KeyError as err:
            tw_description = np.nan
            print('KeyError at index {} twitter:description'.format(index))
            print('continuing...')

        # ---------------------------------------------------------------------------- #
        #                               regular meta tags                              #
        # ---------------------------------------------------------------------------- #

        # keywords, description
        description = soup.find('meta', attrs={'name':"description"})
        try:
            description = description['content'] if description else np.nan
        except KeyError as err:
            description = np.nan
            print('KeyError at index {} description'.format(index))
            print('continuing...')
        
        keywords = soup.find('meta', attrs={'name':"keywords"})
        try:
            keywords = keywords['content'] if keywords else np.nan
        except KeyError as err:
            keywords = np.nan
            print('KeyError at index {} keywords'.format(index))
            print('continuing...')

        dict_['index'].append(index)
        dict_['label'].append(label)
        dict_['site'].append(site)
        dict_['og:title'].append(og_title)
        dict_['og:description'].append(og_description)
        dict_['og:keywords'].append(og_keywords)
        dict_['twitter:description'].append(tw_description)
        dict_['twitter:title'].append(tw_title)
        dict_['twitter:keywords'].append(tw_keywords)
        dict_['keywords'].append(keywords)
        dict_['description'].append(description)

df = pd.DataFrame(dict_)


  0%|          | 0/2 [00:00<?, ?it/s]

KeyError at index 1008 twitter:title
continuing...
KeyError at index 1008 twitter:description
continuing...
KeyError at index 102 description
continuing...
KeyError at index 263 keywords
continuing...
KeyError at index 482 keywords
continuing...
KeyError at index 489 twitter:title
continuing...
KeyError at index 489 twitter:description
continuing...
KeyError at index 498 keywords
continuing...
KeyError at index 500 keywords
continuing...
KeyError at index 57 description
continuing...
KeyError at index 579 og:description
continuing...
KeyError at index 579 twitter:description
continuing...
KeyError at index 581 keywords
continuing...
KeyError at index 6 keywords
continuing...
KeyError at index 633 og:description
continuing...
KeyError at index 700 description
continuing...
KeyError at index 80 description
continuing...
KeyError at index 82 description
continuing...
KeyError at index 909 title
continuing...
KeyError at index 909 twitter:title
continuing...


100%|██████████| 2/2 [04:15<00:00, 127.80s/it]


## old:

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from pathlib import Path
import numpy as np
import pickle
from tqdm import tqdm

# Get url keys
with open('real_url_key.pkl', 'rb') as f:
    real_url_key = pickle.load(f)
f.close()
with open('fake_url_key.pkl', 'rb') as f:
    fake_url_key = pickle.load(f)
f.close()

directory1 = Path.cwd() / 'HTML' / 'real'
directory2 = Path.cwd() / 'HTML' / 'fake'

directory = [directory1, directory2]
dict_ = {'index' : [],
        'label' : [],
        'site' : [],
        'og:title' : [],
        'keywords': [],
        'description' : []}

for dir_ in tqdm(directory):
    for html in os.listdir(dir_):
        # Open html file as soup object
        with open(str(dir_) + '/' + html, encoding='utf8') as f:
            soup = BeautifulSoup(f, 'html.parser')
        f.close()

        # metadata
        index = int(str(html).split('.')[0])
        label = str(dir_)[-4:]
        site = real_url_key[index] if str(dir_)[-4:] == 'real' else fake_url_key[index]
        
        # og:title
        title = soup.find("meta", property="og:title")
        try:
            og_title = title['content'] if title else np.nan
        except KeyError as err:
            og_title = np.nan
            print('KeyError at index {} title'.format(index))
            print('continuing...')

        # keywords, description
        meta = soup.find_all('meta')
        keywords = np.nan
        description = np.nan
        for tag in meta:
            if 'name' in tag.attrs.keys() and tag.attrs['name'].strip().lower() in ['description']:
                try:
                    description = tag.attrs['content']
                except KeyError as err:
                    print('KeyError at index {} description'.format(index))
                    print('continuing...')
                    pass
            elif 'name' in tag.attrs.keys() and tag.attrs['name'].strip().lower() in ['keywords']:
                try:
                    keywords = tag.attrs['content']
                except KeyError as err:
                    print('KeyError at index {} keywords'.format(index))
                    print('continuing...')
                    pass

        dict_['index'].append(index)
        dict_['label'].append(label)
        dict_['site'].append(site)
        dict_['og:title'].append(og_title)
        dict_['keywords'].append(keywords)
        dict_['description'].append(description)

df = pd.DataFrame(dict_)


In [4]:
df.to_excel("html_data.xlsx", sheet_name="sheet1")

In [41]:
df.to_excel("html_data_expanded.xlsx", sheet_name="sheet1")